In [ ]:
import pandas as pd
import numpy as np

# Function to display full dataset with borders
def display_full(df, title):
    print(f"\n{'='*50}")
    print(f"{title.upper():^50}")
    print(f"{'='*50}")
    with pd.option_context('display.max_rows', None, 
                         'display.max_columns', None,
                         'display.width', 1000):
        print(df.to_string())
    print(f"\nMissing values:\n{df.isna().sum()}")
    print(f"\nData shape: {df.shape}")
    print(f"{'='*50}\n")
    
    # Load the datasets
print("\n\033[1m" + "="*50)
print("LOADING ORIGINAL SALES DATA".center(50))
print("="*50 + "\033[0m")
sales_df = pd.read_csv('Sales.csv')
display_full(sales_df, "Original Sales Dataset")


           LOADING ORIGINAL SALES DATA            

              ORIGINAL SALES DATASET              
   Order ID  Customer Name   Order Date   Product  Quantity  Unit Price  Total Revenue
0      1001       John Doe   01/01/2024  Widget A      10.0        25.0          250.0
1      1002     Jane Smith   01/02/2024  Widget B       5.0        40.0          200.0
2      1003            NaN  2024/01/03'  Widget A       NaN        25.0            NaN
3      1004  Alice Johnson   04/01/2024  Widget C       3.0         NaN          210.0
4      1005      Bob Brown  2024/01/05'  Widget B      10.0        40.0          400.0
5      1006       John Doe   06/01/2024  Widget A       4.0        25.0          100.0
6      1001       John Doe   01/01/2024  Widget A      10.0        25.0          250.0
7      1007     Jane Smith   07/01/2024  Widget C      -6.0        70.0         -420.0

Missing values:
Order ID         0
Customer Name    1
Order Date       0
Product          0
Quantity         1
U

In [15]:
# 1. Handle missing values
print("\n\033[1m" + "="*50)
print("STEP 1: HANDLING MISSING VALUES".center(50))
print("="*50 + "\033[0m")

print("\n\033[94mFilling missing Customer Names with 'Unknown'...\033[0m")
print(f"Before - Missing Customer Names: {sales_df['Customer Name'].isna().sum()}")
sales_df['Customer Name'] = sales_df['Customer Name'].fillna('Unknown')
print(f"After - Missing Customer Names: {sales_df['Customer Name'].isna().sum()}")

print("\n\033[94mFilling missing Quantity with 1...\033[0m")
print(f"Before - Missing Quantity values: {sales_df['Quantity'].isna().sum()}")
sales_df['Quantity'] = sales_df['Quantity'].fillna(1)
print(f"After - Missing Quantity values: {sales_df['Quantity'].isna().sum()}")

print("\n\033[94mCalculating missing Unit Prices from Total Revenue...\033[0m")
mask = (sales_df['Unit Price'].isna()) & (sales_df['Total Revenue'].notna()) & (sales_df['Quantity'] > 0)
print(f"Found {mask.sum()} rows where Unit Price can be calculated")
sales_df.loc[mask, 'Unit Price'] = sales_df.loc[mask, 'Total Revenue'] / sales_df.loc[mask, 'Quantity']

print("\n\033[94mFilling remaining missing Unit Prices with median...\033[0m")
print(f"Before - Missing Unit Price values: {sales_df['Unit Price'].isna().sum()}")
sales_df['Unit Price'] = sales_df['Unit Price'].fillna(sales_df['Unit Price'].median())
print(f"After - Missing Unit Price values: {sales_df['Unit Price'].isna().sum()}")

print("\n\033[94mRecalculating Total Revenue where needed...\033[0m")
sales_df['Total Revenue'] = sales_df['Quantity'] * sales_df['Unit Price']

display_full(sales_df, "After Missing Value Treatment")


         STEP 1: HANDLING MISSING VALUES          

Filling missing Customer Names with 'Unknown'...
Before - Missing Customer Names: 1
After - Missing Customer Names: 0

Filling missing Quantity with 1...
Before - Missing Quantity values: 1
After - Missing Quantity values: 0

Calculating missing Unit Prices from Total Revenue...
Found 1 rows where Unit Price can be calculated

Filling remaining missing Unit Prices with median...
Before - Missing Unit Price values: 0
After - Missing Unit Price values: 0

Recalculating Total Revenue where needed...

          AFTER MISSING VALUE TREATMENT           
   Order ID  Customer Name   Order Date   Product  Quantity  Unit Price  Total Revenue
0      1001       John Doe   01/01/2024  Widget A      10.0        25.0          250.0
1      1002     Jane Smith   01/02/2024  Widget B       5.0        40.0          200.0
2      1003        Unknown  2024/01/03'  Widget A       1.0        25.0           25.0
3      1004  Alice Johnson   04/01/2024  Widg

In [16]:
# 2. Fix inconsistent date formats
print("\n\033[1m" + "="*50)
print("STEP 2: FIXING DATE FORMATS".center(50))
print("="*50 + "\033[0m")

print("\n\033[94mRemoving apostrophes from dates...\033[0m")
sales_df['Order Date'] = sales_df['Order Date'].str.replace("'", "")

print("\n\033[94mConverting to datetime...\033[0m")
print(f"Invalid dates before conversion: {sales_df['Order Date'].isna().sum()}")
sales_df['Order Date'] = pd.to_datetime(sales_df['Order Date'], errors='coerce')
print(f"Invalid dates after conversion: {sales_df['Order Date'].isna().sum()}")

display_full(sales_df, "After Date Fixing")


           STEP 2: FIXING DATE FORMATS            

Removing apostrophes from dates...

Converting to datetime...
Invalid dates before conversion: 0
Invalid dates after conversion: 2

                AFTER DATE FIXING                 
   Order ID  Customer Name Order Date   Product  Quantity  Unit Price  Total Revenue
0      1001       John Doe 2024-01-01  Widget A      10.0        25.0          250.0
1      1002     Jane Smith 2024-01-02  Widget B       5.0        40.0          200.0
2      1003        Unknown        NaT  Widget A       1.0        25.0           25.0
3      1004  Alice Johnson 2024-04-01  Widget C       3.0        70.0          210.0
4      1005      Bob Brown        NaT  Widget B      10.0        40.0          400.0
5      1006       John Doe 2024-06-01  Widget A       4.0        25.0          100.0
6      1001       John Doe 2024-01-01  Widget A      10.0        25.0          250.0
7      1007     Jane Smith 2024-07-01  Widget C      -6.0        70.0         -420.0

In [17]:
# 3. Remove duplicate rows
print("\n\033[1m" + "="*50)
print("STEP 3: REMOVING DUPLICATES".center(50))
print("="*50 + "\033[0m")

print(f"\nShape before removing duplicates: {sales_df.shape}")
print(f"Duplicate Order ID-Product combinations: {sales_df.duplicated(subset=['Order ID', 'Product']).sum()}")
sales_df = sales_df.drop_duplicates(subset=['Order ID', 'Product'], keep='first')
print(f"Shape after removing duplicates: {sales_df.shape}")

display_full(sales_df, "After Duplicate Removal")



           STEP 3: REMOVING DUPLICATES            

Shape before removing duplicates: (8, 7)
Duplicate Order ID-Product combinations: 1
Shape after removing duplicates: (7, 7)

             AFTER DUPLICATE REMOVAL              
   Order ID  Customer Name Order Date   Product  Quantity  Unit Price  Total Revenue
0      1001       John Doe 2024-01-01  Widget A      10.0        25.0          250.0
1      1002     Jane Smith 2024-01-02  Widget B       5.0        40.0          200.0
2      1003        Unknown        NaT  Widget A       1.0        25.0           25.0
3      1004  Alice Johnson 2024-04-01  Widget C       3.0        70.0          210.0
4      1005      Bob Brown        NaT  Widget B      10.0        40.0          400.0
5      1006       John Doe 2024-06-01  Widget A       4.0        25.0          100.0
7      1007     Jane Smith 2024-07-01  Widget C      -6.0        70.0         -420.0

Missing values:
Order ID         0
Customer Name    0
Order Date       2
Product          

In [18]:
# 4. Fix wrong data - Enhanced Negative Value Handling
print("\n\033[1m" + "="*50)
print("STEP 4: FIXING DATA ERRORS".center(50))
print("="*50 + "\033[0m")

# Store original shape for comparison
original_shape = sales_df.shape

# 1. First display ALL problematic rows together
neg_qty_mask = sales_df['Quantity'] < 0
neg_rev_mask = sales_df['Total Revenue'] < 0
combined_mask = neg_qty_mask | neg_rev_mask

if combined_mask.any():
    print("\n\033[91mPROBLEMATIC ROWS FOUND:\033[0m")
    print(sales_df[combined_mask].to_string())
    
    # 2. Handle negative quantities
    if neg_qty_mask.any():
        print("\n\033[93mNEGATIVE QUANTITIES DETECTED:\033[0m")
        print(sales_df[neg_qty_mask][['Order ID', 'Product', 'Quantity']].to_string())
        
        print("\n\033[92mCORRECTING NEGATIVE QUANTITIES...\033[0m")
        sales_df.loc[neg_qty_mask, 'Quantity'] = sales_df.loc[neg_qty_mask, 'Quantity'].abs()
        print("Correction applied to quantities")
    
    # 3. Handle negative revenue
    if neg_rev_mask.any():
        print("\n\033[93mNEGATIVE REVENUE DETECTED:\033[0m")
        print(sales_df[neg_rev_mask][['Order ID', 'Product', 'Total Revenue']].to_string())
        
        print("\n\033[92mREMOVING ROWS WITH NEGATIVE REVENUE...\033[0m")
        sales_df = sales_df[~neg_rev_mask]
        print(f"Removed {neg_rev_mask.sum()} rows. New shape: {sales_df.shape}")
    
    # 4. Show final status of problematic rows
    print("\n\033[92mFINAL STATUS OF PROBLEMATIC ROWS:\033[0m")
    if neg_qty_mask.any() and not neg_rev_mask.any():
        print("All negative quantities corrected:")
        print(sales_df.loc[neg_qty_mask, ['Order ID', 'Product', 'Quantity']].to_string())
    elif not combined_mask.any():
        print("All problematic rows have been resolved")
else:
    print("\nNo data issues found (no negative quantities or revenue)")

# Save cleaned data
sales_df.to_csv('Sales_cleaned.csv', index=False)
print("\n\033[92mSaved cleaned data to 'Sales_cleaned.csv'\033[0m")


            STEP 4: FIXING DATA ERRORS            

PROBLEMATIC ROWS FOUND:
   Order ID Customer Name Order Date   Product  Quantity  Unit Price  Total Revenue
7      1007    Jane Smith 2024-07-01  Widget C      -6.0        70.0         -420.0

NEGATIVE QUANTITIES DETECTED:
   Order ID   Product  Quantity
7      1007  Widget C      -6.0

CORRECTING NEGATIVE QUANTITIES...
Correction applied to quantities

NEGATIVE REVENUE DETECTED:
   Order ID   Product  Total Revenue
7      1007  Widget C         -420.0

REMOVING ROWS WITH NEGATIVE REVENUE...
Removed 1 rows. New shape: (6, 7)

FINAL STATUS OF PROBLEMATIC ROWS:

Saved cleaned data to 'Sales_cleaned.csv'


In [19]:
# Final Output Display - Run After Cleaning
def display_full(df, title):
    print(f"\n{'='*50}")
    print(f"{title.upper():^50}")
    print(f"{'='*50}")
    with pd.option_context('display.max_rows', None, 
                         'display.max_columns', None,
                         'display.width', 1000):
        print(df.to_string())
    print(f"\nMissing values:\n{df.isna().sum()}")
    print(f"\nData shape: {df.shape}")
    print(f"{'='*50}\n")

print("\n\033[1m" + "="*50)
print("FINAL CLEANED SALES DATA".center(50))
print("="*50 + "\033[0m")
display_full(sales_df, "Cleaned Sales Dataset")
print("\033[92mData cleaning complete!\033[0m")


             FINAL CLEANED SALES DATA             

              CLEANED SALES DATASET               
   Order ID  Customer Name Order Date   Product  Quantity  Unit Price  Total Revenue
0      1001       John Doe 2024-01-01  Widget A      10.0        25.0          250.0
1      1002     Jane Smith 2024-01-02  Widget B       5.0        40.0          200.0
2      1003        Unknown        NaT  Widget A       1.0        25.0           25.0
3      1004  Alice Johnson 2024-04-01  Widget C       3.0        70.0          210.0
4      1005      Bob Brown        NaT  Widget B      10.0        40.0          400.0
5      1006       John Doe 2024-06-01  Widget A       4.0        25.0          100.0

Missing values:
Order ID         0
Customer Name    0
Order Date       2
Product          0
Quantity         0
Unit Price       0
Total Revenue    0
dtype: int64

Data shape: (6, 7)

Data cleaning complete!
